In [ ]:
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
import os

import scripts
from scripts import sbi_tools
from scripts import plot_utils

%load_ext autoreload
%autoreload 2

%matplotlib inline
mpl.pyplot.style.use('default')
mpl.pyplot.close('all')

font, rcnew = plot_utils.matplotlib_default_config()
mpl.rc('font', **font)
mpl.pyplot.rcParams.update(rcnew)
mpl.pyplot.style.use('tableau-colorblind10')
%config InlineBackend.figure_format = 'retina'

N_threads = sbi_tools.set_N_threads(6)

In [ ]:
tag_emuPk = '_5param'

fn_emuPk = f'../data/emuPks/emuPks{tag_emuPk}.npy'
fn_emuPk_params = f'../data/emuPks/emuPks_params{tag_emuPk}.txt'
fn_emuk = f'../data/emuPks/emuPks_k{tag_emuPk}.txt'

Pk = np.load(fn_emuPk)
theta = np.genfromtxt(fn_emuPk_params, delimiter=',', names=True)
param_names = theta.dtype.names
# from tuples to 2d array
theta = np.array([list(tup) for tup in theta])
kk = np.genfromtxt(fn_emuk)

In [ ]:
mask = np.full(len(kk), True)

In [ ]:
print(Pk.shape, theta.shape)

In [ ]:
# TRY WITH SUBSET
n_samples = 100
Pk = Pk[:n_samples]
theta = theta[:n_samples]

In [ ]:
n_tot = theta.shape[0]
n_params = theta.shape[1]

In [ ]:
fig, ax = mpl.pyplot.subplots(figsize=(8, 6))
for iLH in range(n_tot):
    ax.loglog(kk[mask], Pk[iLH][mask])

ax.set_xlabel(r'$k \,\, [h \,\, {\rm Mpc}^{-1}]$', fontsize=23)
ax.set_ylabel(r'$P(k) \,\, [h^{-3} \,\, {\rm Mpc}^3]$', fontsize=23)

mpl.pyplot.tight_layout()
mpl.pyplot.show()

In [ ]:
#n_biasmodels = len(biases_vec)
n_biasmodels = 0
n_cosmos = n_params
print(n_biasmodels, n_cosmos)

In [ ]:
### don't include bias parameters
#theta = np.repeat(theta_cosmo, 10, axis=0)

### include b1
# b1_set = biases_vec[:,0]
# b1s = np.expand_dims(np.tile(b1_set, n_cosmos), axis=1)
# theta = np.concatenate((np.repeat(theta_cosmo, n_biasmodels, axis=0), b1s), axis=1)

In [ ]:
#iLH = 0
# for i in range(n_biasmodels):
#     plt.loglog(kk, Pk[i], label=f'b1 = {theta[i,-1]:.2f}')

# plt.legend(fontsize=8)
# ax = plt.gca()

# ax.set_xlabel(r'$k \,\, [h \,\, {\rm Mpc}^{-1}]$', fontsize=23)
# ax.set_ylabel(r'$P(k) \,\, [h^{-3} \,\, {\rm Mpc}^3]$', fontsize=23)

# mpl.pyplot.tight_layout()
# mpl.pyplot.show()

In [ ]:
# rng = np.random.default_rng(seed=533)
# n_tot = len(Pk)

# i_blocks = np.arange(n_tot).reshape(n_cosmos, n_biasmodels)

# i_shuffle_blocks = rng.choice(np.arange(n_cosmos), replace=False, size=n_cosmos)

# i_shuffle = i_blocks[i_shuffle_blocks].flatten()

# Pk = Pk[i_shuffle]
# theta = theta[i_shuffle]


In [ ]:
train_val_split = int(theta.shape[0]*0.8)
#train_val_split = int(n_biasmodels*round(theta.shape[0]*0.99/n_biasmodels))

theta_train = theta[:train_val_split]
theta_test = theta[train_val_split:]
print(theta_train.shape, theta_test.shape)

Pk_train = Pk[:train_val_split]
Pk_test = Pk[train_val_split:]

mask = np.all(Pk_train>0, axis=0)
Pk_train = Pk_train[:,mask]
Pk_test = Pk_test[:,mask]
k = kk[mask]

In [ ]:
fig, ax = mpl.pyplot.subplots(1,1, figsize=(7,5))
fontsize = 24
fontsize1 = 18

alpha = 1

tmp_Pk_plot = Pk_train
tmp_Pk_plot = tmp_Pk_plot[np.random.choice(tmp_Pk_plot.shape[0], tmp_Pk_plot.shape[0], replace=False)].T
ax.plot(np.log10(k), np.log10(tmp_Pk_plot), c='royalblue', alpha=alpha, lw=0.5)

tmp_Pk_plot = Pk_test
tmp_Pk_plot = tmp_Pk_plot[np.random.choice(tmp_Pk_plot.shape[0], tmp_Pk_plot.shape[0], replace=False)].T
ax.plot(np.log10(k), np.log10(tmp_Pk_plot), c='k', alpha=alpha, lw=0.5)
    
mpl.pyplot.tight_layout()
mpl.pyplot.show()

In [ ]:
dict_bounds = {}
for pp, param_name in enumerate(param_names):
    dict_bounds[param_name] = [np.min(theta[:,pp]), np.max(theta[:,pp])]

In [ ]:
class Scaler:

    def __init__(self):
          pass
        
    def fit(self, x_train):
        self.x_train_min = np.min(x_train)
        self.x_train_max = np.max(x_train)
           
    def scale(self, x):
        log_x = np.log10(x)
        log_x_norm = (log_x - np.log10(self.x_train_min)) / (np.log10(self.x_train_max) - np.log10(self.x_train_min))
        return log_x_norm
    
    def unscale(self, x_scaled):
        x = x_scaled * (np.log10(self.x_train_max) - np.log10(self.x_train_min)) + np.log10(self.x_train_min)
        return 10**x  

In [ ]:
scaler = Scaler()
scaler.fit(Pk_train)
Pk_train_scaled = scaler.scale(Pk_train)
Pk_test_scaled = scaler.scale(Pk_test)

In [ ]:
print(np.min(Pk_train), np.max(Pk_train))
print(np.min(Pk_train_scaled), np.max(Pk_train_scaled))

print(np.min(Pk_test), np.max(Pk_test))
print(np.min(Pk_test_scaled), np.max(Pk_test_scaled))

In [ ]:
print(Pk_train.shape)
print(theta_train.shape)

In [ ]:
inference, posterior = sbi_tools.train_model(
    theta_train,
    Pk_train_scaled,
    prior= sbi_tools.get_prior(dict_bounds),
    training_batch_size=16,
    validation_fraction=0.2
)

In [ ]:

#idx_train_check = rng.choice(np.arange(len(theta_train)))
idx_train_check = 42

print(idx_train_check)
theta_train_check = np.array([theta_train[idx_train_check]])
print(theta_train_check)
Pk_train_scaled_check = np.array([Pk_train_scaled[idx_train_check]])

In [ ]:
# ------------------ posterior inference ------------------ #

# norm_xx_test = scaler.transform(xx_test)

inferred_theta_train_check = sbi_tools.sample_posteriors_theta_test(
    posterior,
    Pk_train_scaled_check,
    dict_bounds,
    N_samples=1000
)

# ------------------ rank stats ------------------ #

ranks_train_check = sbi_tools.compute_ranks(theta_train_check, inferred_theta_train_check)

In [ ]:
param_labels = {'omega_cold': r'$\Omega_\mathrm{m}$',
                'sigma8_cold': r'$\sigma_{8}$',
                'hubble': r'$h$',
                'ns': r'$n_\mathrm{s}$',
                'omega_baryon': r'$\Omega_\mathrm{b}$',}
custom_titles = [param_labels[param_name] for param_name in param_names]
# custom_titles = [
#     r'$\Omega_\mathrm{m}$',
#     r'$\Omega_\mathrm{b}$',
#     r'$h$',
#     r'$n_\mathrm{s}$',
#     r'$\sigma_{8}$',
#     r'$b_1$'
# ]

In [ ]:
N_examples = 1

colors = plot_utils.get_N_colors(N_examples, mpl.colormaps['prism'])
for ii_sample in range(N_examples):
    fig, axs = plot_utils.corner_plot(
        theta_train_check[ii_sample],
        inferred_theta_train_check[ii_sample],
        custom_titles,
        dict_bounds,
        color_infer=colors[ii_sample]
    )
    mpl.pyplot.show()
    
#fig.save("popopo.png")

In [ ]:
# posterior_sample = posterior.sample((1,))
# print(posterior_sample)

In [ ]:
# ------------------ posterior inference ------------------ #

# norm_xx_test = scaler.transform(xx_test)

inferred_theta_test = sbi_tools.sample_posteriors_theta_test(
    posterior,
    Pk_test_scaled,
    dict_bounds,
    N_samples=1000
)

# ------------------ rank stats ------------------ #

ranks = sbi_tools.compute_ranks(theta_test, inferred_theta_test)

In [ ]:
# tmp_inferred_xx_test = sbi_data_utils.compute_baccoemu_predictions_batch(
#     inferred_theta_train_check[indexes],
#     list(dict_bounds.keys())
# )

# fig, ax, ax_res = plot_utils.plot_xx_from_sampled_posteriors(xx_test[indexes], tmp_inferred_xx_test, kk)
# plt.show()

In [ ]:
N_examples = 1

colors = plot_utils.get_N_colors(N_examples, mpl.colormaps['prism'])
for ii_sample in range(N_examples):
    fig, axs = plot_utils.corner_plot(
        theta_test[ii_sample],
        inferred_theta_test[ii_sample],
        custom_titles,
        dict_bounds,
        color_infer=colors[ii_sample]
    )
    mpl.pyplot.show()
    
#fig.save("popopo.png")

In [ ]:
fig, axs = plot_utils.plot_parameter_prediction_vs_truth(inferred_theta_test, theta_test, custom_titles)
plt.tight_layout()
plt.subplots_adjust(wspace=0.6)
plt.show()

In [ ]:
fig, axs = plot_utils.plot_rank_statistcis(ranks, inferred_theta_test.shape[1], custom_titles)
plt.tight_layout()
plt.subplots_adjust(wspace=0.05)
plt.show()